# Anregung des Einmassenschwingers (EMS) mit verschiedenen Signalen

### Importieren der Bibliotheken

In [1]:
# geladene Bibliotheken und Funktionen
import numpy as np
from sympy import *
from sympy.physics.vector import *
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
from matplotlib import colors as mcolors
from ipywidgets import interactive, Layout

# Hilfsfunktion
def odeToVectorField(odes, dep_vars, indep_var):
    """Überführt Bewegungsgleichungen (a.k.a. ODE-System 2. Ordnung) in ODE-System 1. Ordnung (für numerische Solver).
    
    Ausgabe: 
    1. neue symbolische Variablen x1, ..., xN (generalisierte Koordinaten, dann generalisierte Geschwindigkeiten)
    2. rechte Seite als ODE-System 1. Ordnung"""
    if type(odes) != type(list()):
        odes = [odes]
        dep_vars = [dep_vars]
    rhs = Array([*diff(Array(dep_vars),indep_var), *solve(odes,diff(Array(dep_vars),indep_var,indep_var)).values()])
    old = list(np.array([dep_vars, diff(Array(dep_vars),indep_var)]).flatten())
    new = symbols('x1:'+str(len(old)+1))
    return [new, rhs.subs(dict(zip(old,new)))]

### Einleitung
In diesem Notebook visualisieren wir die Lösung der Bewegungsgleichung des Einmassenschwingers (EMS) für verschiedene Anregungen $F$. Erinnerung: Die Bewegungsgleichung des EMS lautet

$$\ddot{x} + 2D \omega_0 \dot{x} + \omega_0^2 x = \frac{F(t)}{m} \, .$$

Um die Bewegungsgleichung eindeutig zu lösen, d.h. $x(t)$ für alle $t$ in einem gewünschten Zeitintervall zu erhalten, benötigt man noch die beiden Anfangsbedingungen (ABen) $x(t=0) = x_0$ und $\dot{x}(t=0) = v_0$.

In [2]:
m, D, omega0, t = symbols('m, D, omega0, t')
x = Function('x')(t)
F = Function('F')(t)

bewegungsGl = x.diff(t,t) + 2*D*omega0*x.diff() + omega0**2*x - F/m
bewegungsGl

2*D*omega0*Derivative(x(t), t) + omega0**2*x(t) + Derivative(x(t), (t, 2)) - F(t)/m

## Freie Schwingungen (Unterkritische Dämpfung: $0\leq D<1$)

Wir visualisieren $x$ zunächst für den Fall *freier* Schwingungen, d.h. $F \equiv 0$. Es gibt somit keine Anregung und allein die ABen stehen als freie Parameter zur Verfügung. Im Beispiel sind $x_0 = 1 \text{ m}$ und $v_0 = 0 \text{ m/s}$.

In [3]:
# Anfangsbedingungen
x0 = 1
v0 = 0

# Zeit, t
tsteps = np.linspace(0,20,1000)
    
def update(omega0,D):    
    # gedämpfte Eigenkreisfrequenz
    omega_d = omega0*np.sqrt(1-D**2)

    # Integrationskonstanten
    A = x0
    B = (v0 + D*x0*omega0)/omega_d
    
    # Position, x
    pos = np.exp(-D*omega0*tsteps)*(A*np.cos(omega_d*tsteps) + B*np.sin(omega_d*tsteps))
 
    plt.figure(figsize=(9,6))
    plt.plot(tsteps,pos)
    plt.xlim(*tsteps[[0,-1]])
    plt.ylim([-1.2,1.2])
    plt.xlabel('$t$ in s',size=15)
    plt.ylabel('$x$ in m',size=15)
    plt.grid(True)
    plt.show()
    

interactive_plot = interactive(update, omega0=(1,10,.01), D=(0,.99,.01))
widget1 = interactive_plot.children[0]
widget2 = interactive_plot.children[1]
widget1.layout = Layout(width='400px', align_items='center')
widget2.layout = Layout(width='400px', align_items='center')
widget1.description = '$\omega_0$ [rad/s]'
widget2.description = '$D$ [1]'
#widget1.continuous_update = True
#widget2.continuous_update = True
output = interactive_plot.children[-1]
interactive_plot.layout = Layout(width='570px',align_items='center')
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='$\\omega_0$ [rad/s]', layout=Layout(align_items='cen…

Mit den beiden Schiebereglern können die Eigenkreisfrequenz $\omega_0$ und der Dämpfungsgrad $D$ des EMS eingestellt werden.

## Erzwungene Schwingungen 
### Sprung bei $t=10$s
Wir betrachten als nächstes die Auswirkungen eines Sprungs in der Kraftanregung von $0 \text{ N}$ auf $1 \text{ N}$ zum Zeitpunkt $t=10$.

In [4]:
params = {m: .3}
var, system = odeToVectorField(bewegungsGl, x, t)

rhs = lambdify([t, var, F, omega0, D], system.subs(params))
anregung = np.vectorize(lambda t: 0 if t<10 else 1)

# Zeitspanne für welche die Bewegungsgleichungen gelöst werden soll
tspan = [0,20] # in s
# Anfangsbedingungen 
ABen = [0, 0]  # in m, m/s


def update(omega0,D):
    sol=solve_ivp(lambda t, var: rhs(t,var,anregung(t),omega0,D),tspan,ABen,rtol=1e-9,atol=1e-12,t_eval=np.linspace(*tspan,1000))
    
    plt.figure(figsize=(9,6))
    plt.plot(sol.t,sol.y[0,:])
    plt.xlim(*tspan)
    plt.ylim([0,1])
    plt.xlabel('$t$ in s',size=15)
    plt.ylabel('$x$ in m',size=15)
    plt.grid(True)
    plt.show()
    
    
interactive_plot = interactive(update, omega0=(2.5,10,.01), D=(0,.99,.01))
widget1 = interactive_plot.children[0]
widget2 = interactive_plot.children[1]
widget1.layout = Layout(width='400px', align_items='center')
widget2.layout = Layout(width='400px', align_items='center')
widget1.description = '$\omega_0$ [rad/s]'
widget2.description = '$D$ [1]'
#widget1.continuous_update = True
#widget2.continuous_update = True
output = interactive_plot.children[-1]
interactive_plot.layout = Layout(width='570px',align_items='center')
interactive_plot

interactive(children=(FloatSlider(value=6.25, description='$\\omega_0$ [rad/s]', layout=Layout(align_items='ce…

### Kosinusanregung: $F(t) = F_0 \cos(\Omega t)$

Um die Position $x$ zu zeichnen, brauchen wir die homogene Lösung $x_h$ und die partikuläre Lösung $x_p$. Es gilt 

$$ 
\begin{align} 
x_h(t) &= e^{-D \omega_0 t} (A \cos(\omega_d t) + B \sin(\omega_d t)) \, , \\
x_p(t) &= \kappa \alpha(\eta) F_0 \cos(\Omega t - \phi(\eta)) \, .
\end{align}$$

Außerdem wissen wir dass $x = x_h + x_p$ gilt. In der homogenen Lösung $x_h$ und somit in der Gesamtlösung $x$ stehen die noch unbekannten Integrationskonstanten $A, B$, die aus den Anfangsbedingungen $x(t=0)=x_0$ und $\dot{x}(t=0)=v_0$ bestimmt werden können. Wir verwenden `sympy` um diese beiden Bedingungen nach $A$ und $B$ aufzulösen. Im Anschluss daran definieren wir die numerischen Funktionen `pos` und `anregung`, die $x$ und $F$ entsprechen und zeichnen diese in ein Diagramm.

In [11]:
params = {m: .3, D: .08, omega0: 3}

F0 = symbols('F0')
params[F0] = 1

kappa, omega_d = symbols('kappa, omega_d')
Omega, eta = symbols('Omega, eta')
x0, v0 = symbols('x0, v0')
A, B = symbols('A, B')
alpha, phi = symbols('alpha, phi')

xh = exp(-D*omega0*t)*(A*cos(omega_d*t)+B*sin(omega_d*t)) # homogene Lösung
xp = kappa*alpha*F0*cos(Omega*t-phi)                      # partikuläre Lösung
x = xh + xp                                               # Gesamtlösung 

# Integrationskonstanten A, B der homogenen Lösung
intKonstanten = solve([x.subs(t,0) - x0, x.diff(t).subs(t,0) - v0],[A,B])

ampfreqgang = 1/sqrt((1-eta**2)**2 + 4*D**2*eta**2)
phasenfreqgang = atan2(2*D*eta,1-eta**2) 
pos = x.subs(intKonstanten).subs([(alpha,ampfreqgang),(phi,phasenfreqgang)]).subs(params)

pos = lambdify([kappa, omega_d, Omega, eta, x0, v0, t],pos)
anregung = lambdify([F0,Omega,t], F0*cos(Omega*t))


def update(Omega):
    # Systemparameter
    c = m*omega0**2  # Federsteifigkeit
    kappa = float(1/c.subs(params)) # Übertragungskoeffizient 
    omega_d = float((omega0*sqrt(1-D**2)).subs(params)) # gedämpfte Eigenkreisfrequenz

    # Frequenzverhältnis 
    eta = float(Omega/omega0.subs(params))
    
    # Amplitude der Anregung
    f0 = float(F0.subs(params))

    # Anfangsbedingungen 
    x0 = 1 # in m
    v0 = 0 # in m/s
    
    # Zeit
    tspan = [0, 40]
    tsteps = np.linspace(*tspan,1000)
    
    plt.figure(figsize=(9, 6))
    plt.plot(tsteps, anregung(f0,Omega,tsteps), color=mcolors.CSS4_COLORS['lightgray'], label='$F$')
    plt.plot(tsteps, pos(kappa, omega_d, Omega, eta, x0, v0, tsteps), label='$x$')
    plt.xlim(*tspan)
    plt.ylim(-2.2,2.2)
    plt.xlabel('$t$',size=15)
    plt.ylabel('$F$ in N, $x$ in m',size=15)
    plt.legend(fontsize=14)
    plt.show()
    
interactive_plot = interactive(update, Omega=(1,10,.01))
widget = interactive_plot.children[0]
widget.layout = Layout(width='400px', align_items='center')
widget.description = '$\Omega$ [rad/s]'
widget.continuous_update = True
output = interactive_plot.children[-1]
interactive_plot.layout = Layout(width='570px',align_items='center')
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='$\\Omega$ [rad/s]', layout=Layout(align_items='cente…

Durch Bewegung des Schiebereglers kann die Anregerkreisfrequenz $\Omega$ im Bereich von $1 \text{ rad/s}$ bis $10 \text{ rad/s}$ geändert werden. 
Für welches $\Omega$ ist die Amplitude von $x$ im eingeschwungenen Zustand am größten?

### Rechteckanregung: $\DeclareMathOperator{\square}{square} 
F(t) = 1\text{N} \square(\Omega t)$

In [10]:
square = lambda t, Omega: np.sign(np.sin(Omega*t))

def update(Omega):
    plt.figure(figsize=(9, 6))
    rhs = lambdify([t, var, F], system.subs(params))
    anregung = lambda t: square(t,Omega)
    
    # Zeitspanne für welche die Bewegungsgleichungen gelöst werden soll
    tspan = [0,40] # in s
    # Anfangsbedingungen 
    ABen = [1, 0]  # in m, m/s

    sol=solve_ivp(lambda t, var: rhs(t,var, anregung(t)),tspan,ABen,rtol=1e-9,atol=1e-12,t_eval=np.linspace(*tspan,1000))
    plt.plot(sol.t,anregung(sol.t),color=mcolors.CSS4_COLORS['lightgray'])
    plt.plot(sol.t,sol.y[0,:])
    plt.xlim(*tspan)
    plt.ylim(-3,3)
    plt.xlabel('$t$',size=15)
    plt.ylabel('$x(t)$',size=15)
    plt.show()
    
interactive_plot = interactive(update, Omega=(1,7,.01))
widget = interactive_plot.children[0]
widget.layout = Layout(width='400px', align_items='center')
widget.description = '$\Omega$ [rad/s]'
widget.continuous_update = False
output = interactive_plot.children[-1]
interactive_plot.layout = Layout(width='570px',align_items='center')
interactive_plot

interactive(children=(FloatSlider(value=4.0, continuous_update=False, description='$\\Omega$ [rad/s]', layout=…